# ADM-HW4: GROUP #10

#### Xavier Del Giudice, 


|      STUDENT       |   ID    |                 E-mail                  |
|:------------------:|:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  Flavio Mangione   |  |      |
|     Leo Rocci      |  |       |
|       |  |      |

## Import and Install required packages

In [1]:
%pip install pandas numpy scikit-learn matplotlib pyspark plotly kagglehub

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ed/8c/87ddf1fcb55d11f9f847e3c69bb1c6f8e46e2f40ab1a2d2abadb2401b007/pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/d0/06/3d1ff6ed377cb0340baf90487a35f15f9dc1db8e0a07de2bf2c54a8e490f/numpy-2.2.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 667.8 kB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/96/a2/cbfb5743de748d574ffdfd557e9cb29ba4f8b8a3e07836c6c176f713de2f/scikit_learn-1.6.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import numpy as np
import kagglehub

## Import the dataset

In [3]:
# Download the dataset directly from kaggle
path = kagglehub.dataset_download("flashgordon/usa-airport-dataset")

100%|██████████| 40.8M/40.8M [00:02<00:00, 20.5MB/s]

Extracting files...


In [4]:
df = pd.read_csv(os.path.join(path, 'Airports2.csv'))

In [5]:
df.shape

(3606803, 15)

In [6]:
df.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606803 entries, 0 to 3606802
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Origin_airport          object 
 1   Destination_airport     object 
 2   Origin_city             object 
 3   Destination_city        object 
 4   Passengers              int64  
 5   Seats                   int64  
 6   Flights                 int64  
 7   Distance                int64  
 8   Fly_date                object 
 9   Origin_population       int64  
 10  Destination_population  int64  
 11  Org_airport_lat         float64
 12  Org_airport_long        float64
 13  Dest_airport_lat        float64
 14  Dest_airport_long       float64
dtypes: float64(4), int64(6), object(5)
memory usage: 412.8+ MB


In [9]:
df.isnull().sum()

Origin_airport               0
Destination_airport          0
Origin_city                  0
Destination_city             0
Passengers                   0
Seats                        0
Flights                      0
Distance                     0
Fly_date                     0
Origin_population            0
Destination_population       0
Org_airport_lat           6954
Org_airport_long          6954
Dest_airport_lat          6807
Dest_airport_long         6807
dtype: int64

## 1. Flight Network Analysis (Q1)

### Implement a function `analyze_graph_features(flight_network)` that takes the flight network as input and computes the following:
   - Count the number of airports (nodes) and flights (edges) in the graph.
   - Compute the density of the graph using the formula:  $Density = \frac{2 \cdot E}{N \cdot (N - 1)}$
   - Calculate both in-degree and out-degree for each airport and visualize them using histograms.
   - Identify airports with degrees higher than the 90th percentile and list them as "hubs."
   - Determine if the graph is sparse or dense based on its density.

In [ ]:
from functions import flight_network

### Write a function `summarize_graph_features(flight_network)` that generates a detailed report of the graph's features. A summary report needs to include:
   - The number of nodes and edges.
   - The graph density.
   - Degree distribution plots for in-degree and out-degree.
   - A table of identified hubs.

### Now let's dive deeper into the analysis of the dataset. Do the following:
   - Compute total passenger flow between origin and destination cities.
   - Identify and visualize the busiest routes by passenger traffic.
   - Calculate the average passengers per flight for each route and highlight under/over-utilized connections.
   - Create an interactive map visualizing the geographic spread of the flight network.

### **Questions to Address:**

After completing the analysis, answer the following questions:
- Is the graph sparse or dense?
- What patterns do you observe in the degree distribution?
- Which airports are identified as hubs, and why?
- What are the busiest routes in terms of passenger traffic?
- Which routes are under/over-utilized?


## 2. Nodes' Contribution (Q2)

### Implement a function `analyze_centrality(flight_network, airport)` that computes the following centrality measures for a given airport:
   - _Betweenness centrality_: Measures how often a node appears on the shortest paths between other nodes.
   - _Closeness centrality_: Measures how easily a node can access all other nodes in the network.
   - _Degree centrality_: Simply counts the number of direct connections to the node.
   - _PageRank_: Computes the "importance" of a node based on incoming connections and their weights.

### Write a function `compare_centralities(flight_network)` to:
   - Compute and compare centrality values for all nodes in the graph.
   - Plot centrality distributions (histograms for each centrality measure).
   - Return the top 5 airports for each centrality measure.

### Ask LLM (eg. ChatGPT) to suggest alternative centrality measures that might be relevant to this task. How can you check that the results given by the LLM are trustable?

### Implement one of these measures suggested by the LLM, compare its results to the centralities you've already computed, and analyze whether it adds any new insights.

## 3. Finding Best Routes (Q3)

### In this task, you need to implement a function that, given an origin and destination city, determines the best possible route between them. To simplify, the focus will be limited to flights operating on a specific day.

## 4. Airline Network Partitioning (Q4)

### In graph theory, this task is known as a graph disconnection problem. Your goal is to write a function that removes the minimum number of flights between airports to separate the original flight network into two disconnected subgraphs.

## 5. Finding and Extracting Communities (Q5)

### In this task, you are asked to analyze the graph and identify the communities based on the flight network provided. For the airline, the primary focus is on the cities, so your communities should reflect the connectivity between cities through the flights that link them.

### Ask a LLM (ChatGPT, Claude AI, Gemini, Perplexity, etc.) to suggest an alternative algorithm for extracting communities and explain the steps required to implement it. Then, implement this algorithm and compare its results with the current method you've chosen. Discuss the differences in the outcomes and analyze which approach you think is better, providing reasons for your choice.

## Bonus Question - Connected Components on MapReduce 

### In this task, you are required to use PySpark and the MapReduce paradigm to identify the connected components in a flight network graph. The focus should be on airports rather than cities. As you know, a connected component refers to a group of airports where every pair of airports within the group is connected either directly or indirectly.

### Compare the execution time and the results of your implementation with those of the GraphFrames package for identifying connected components. If there is any difference in the results, provide an explanation for why that might occur.

## Algorithmic Question (AQ)

Arya needs to travel between cities using a network of flights. Each flight has a fixed cost (in euros), and she wants to find the cheapest possible way to travel from her starting city to her destination city. However, there are some constraints on the journey:

1. Arya can make at most `k` stops during her trip (this means up to `k+1` flights).
2. If no valid route exists within these constraints, the result should be `-1`.

Given a graph of cities connected by flights, your job is to find the minimum cost for Arya to travel between two specified cities (`src` to `dst`) while following the constraints. 

### Your Task

- **a)** Write a pseudocode that describes the algorithm to find the cheapest route with at most `k` stops. 
  
- **b)** Implement the algorithm in Python and simulate the given test cases.

- **c)** Analyze the algorithm's efficiency. Provide its time complexity and space complexity, and explain whether it is efficient for large graphs (e.g., `n > 100`).

- **d)** Optimize the algorithm to handle larger graphs. Provide an updated pseudocode and analyze the computational complexity of your optimization.

- **e)** Ask LLM (e.g., ChatGPT) for an optimized version of your algorithm. Compare its solution to yours in terms of performance, time complexity, and correctness.

### Examples

#### Example 1

**Input:**

```py
n = 4  
flights = [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]]  
src = 0  
dst = 3  
k = 1  
```
**Output:**

```py
700  
```

**Explanation:**  
Arya's optimal path with at most 1 stop is `0 → 1 → 3`, costing 100 + 600 = 700 euros.  
The path `0 → 1 → 2 → 3` is cheaper but requires 2 stops, which violates the constraints.


#### Example 2

**Input:**
```py
n = 3  
flights = [[0, 1, 100], [1, 2, 100], [0, 2, 500]]  
src = 0  
dst = 2  
k = 1  
```
**Output:**
```py
200  
```
**Explanation:**  
Arya's optimal path with at most 1 stop is `0 → 1 → 2`, costing 100 + 100 = 200 euros.


#### Example 3

**Input:**
```py
n = 3  
flights = [[0, 1, 100], [1, 2, 100], [0, 2, 500]]  
src = 0  
dst = 2  
k = 0  
```
**Output:**
```py
500  
```
**Explanation:**  
Arya cannot make any stops. The only valid route is `0 → 2`, costing 500 euros.

#### Example 4

**Input:**
```py
n = 4  
flights = [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]]  
src = 0  
dst = 3  
k = 2  
```
**Output:**
```py
400  
```
**Explanation:**  
Arya can take `0 → 1 → 3` and `0 → 2 → 3`, however first one is cheaper, costing 400 euros.

#### Example 5

**Input:**
```py
n = 4  
flights = [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]]  
src = 0  
dst = 3  
k = 2  
```
**Output:**
```py
400  
```
**Explanation:**  
Arya can take `0 → 1 → 3` and `0 → 2 → 3` like last example. However we have a tie, so it does not matter the route we take, the cost is still 400.